# Opening the .csv's

### Process
- Look into retrieving db-specific EID for blasted gene
    - Result: Not what I thought it was/will not be of use
- Open all of the .csv's
    - Store in cleverly-named dataframes
    - Make new column for name string slices of just new EID
        - add hit EID from slice of hit results
    - Make new column for the first EID that it is associated with
        - add first EID repetitively to this column
    - Save first EID in new dataframe along with the log2fc
    - For each dataframe, ask if the hit EID is in the first_EIDS's df
        - If yes, store the information for that hit EID (e_value, score, norm_score, first_EID, first_log2fc, hit_log2fc) in new dataframe (model)
        - If no, skip
    - Fit the linear regression model
    - Calculate accuracy
        - For each row in df(model), use model to calculate model_hit_log2fc
        - Calculate mean squared error(model_hit_log2fc, hit_log2fc)

- Make user function
    - input FASTA or EID into BLASTn_gene function
    - slice hit name for hit_EID and input into new column in hits_df
    - for each hit_EID, is EID in first_EID's df?
        - If yes, add log2fc to vector for model along with hit_score
        - If no, skip
    - Use model to calculate predicted log2fc

In [3]:
import math
import numpy as np
import pandas as pd

In [4]:
dna002_4 = pd.read_csv('dna004_2.csv')

In [5]:
dna002_4.head()

,Unnamed: 0,e_value,score,hit_name,norm_score
0,0,0.000000e+00,1331.0,pdb|6MZC|M Human TFIID BC core [Homo sapiens] ...,100.000000
1,1,4.825990e-81,612.0,pdb|6F3T|F Crystal structure of the human TAF5...,45.980466
2,2,7.490840e-28,255.0,pdb|1TAF|A Drosophila Tbp Associated Factors D...,19.158527
3,3,3.481010e-27,257.0,pdb|6HQA|F Molecular structure of promoter-bou...,19.308790
4,4,2.493940e-23,231.0,pdb|6T9I|F cryo-EM structure of transcription ...,17.355372


In [13]:
b = "Hello, World!"
print(b[1:5], b[8:11], ';  of note: include space in string index count')

ello orl ;  of note: include space in string index count


In [14]:
df = dna002_4
column = 'hit_name'
i = 0

slice_test = df[column][0][4:8]
print(slice_test)

6MZC


In [19]:
def add_slice_eid(df):
    '''Slices hit titles to take just the EID and returns df appended column for hit eids'''
    
    # make list of hit EIDs by slicing the hit titles
    eid_slices = []
    for i in range(len(df['hit_name'])):
        slice = df['hit_name'][i][4:8]
        eid_slices.append(slice)
    
    # append new column of hit EIDs
    df['hit_eid'] = pd.DataFrame(eid_slices)
    
    # add column for NASA eid
    for i in range(len(df['hit_name'])):
        
    
    return df

In [20]:
# Test

df = dna002_4
csv_filename = 'dna004_eid.csv'

add_slice_eid(df)

,Unnamed: 0,e_value,score,hit_name,norm_score,hit_eid
0,0,0.000000e+00,1331.0,pdb|6MZC|M Human TFIID BC core [Homo sapiens] ...,100.000000,6MZC
1,1,4.825990e-81,612.0,pdb|6F3T|F Crystal structure of the human TAF5...,45.980466,6F3T
2,2,7.490840e-28,255.0,pdb|1TAF|A Drosophila Tbp Associated Factors D...,19.158527,1TAF
3,3,3.481010e-27,257.0,pdb|6HQA|F Molecular structure of promoter-bou...,19.308790,6HQA
4,4,2.493940e-23,231.0,pdb|6T9I|F cryo-EM structure of transcription ...,17.355372,6T9I
5,5,5.952290e-02,75.0,pdb|3V9R|A Crystal structure of Saccharomyces ...,5.634861,3V9R
6,6,7.907670e-02,76.0,pdb|4KUD|A Crystal structure of N-terminal ace...,5.709992,4KUD
7,7,8.352620e-02,76.0,pdb|4JJN|A Crystal structure of heterochromati...,5.709992,4JJN
8,8,1.736980e-01,74.0,pdb|3VH5|A Crystal structure of the chicken CE...,5.559730,3VH5
9,9,2.051970e-01,72.0,pdb|4NDY|A Human MHF1-MHF2 DNA complex [Homo s...,5.409467,4NDY
